# Overall Strctured Data on Commons metrics

[T252443](https://phabricator.wikimedia.org/T252443)

This set of metrics is aiming to understand growth of structured data over time. Including:
- Number of files with at least one structured data element
- Number of files with license
- Number of files with depicts
- Number of files with captions (en vs. non-en vs. both)
- Total number of content pages on Commons (to calculate % in Superset)



### Notes

We aggregate metrics on a monthly basis using data from the `structured_data.commons_entity` table, which is a conversion of the commonswiki structured-data entities JSON dumps in parquet. 
`commons_entity` table has the data back to 2021-12-27 and updated weekly. We take the last snapshot of every month to do the monthly aggregation. 

Captions stored in `lables` field, and each language stored as a key. Example: {"fr":"\"bois fossile : Madagascar\""}.
Entity properties like depicts(P180) and license(P275) are in `statements` field. 

In [1]:
import datetime as dt
import pandas as pd
import numpy as np

from wmfdata import hive, spark

You are using wmfdata v1.3.1, but v1.3.3 is available.

To update, run `pip install --upgrade git+https://github.com/wikimedia/wmfdata-python.git@release --ignore-installed`.

To see the changes, refer to https://github.com/wikimedia/wmfdata-python/blob/release/CHANGELOG.md


### Configuration Timestamps

In [2]:
wmf_snapshot = '2021-12'
entity_snapshot = '2021-12-27'

### Aggregation Tables


We define a set of tables in the Data Lake for aggregation of results.


In [2]:
overall_table = 'cchen_sd.sdc_count_overall'

In [3]:
create_table_query = '''
CREATE TABLE IF NOT EXISTS {table_name} (
    month DATE COMMENT "the month of the aggregated sdc total counts",
    commons_total_count BIGINT COMMENT "total content pages on wiki commons",
    sdc_total_count BIGINT COMMENT "number of files with at least one structured data element",
    caption_total_count BIGINT COMMENT "number of files with at least one caption",
    en_caption_count BIGINT COMMENT "number of files with english caption",
    non_en_caption_count BIGINT COMMENT "number of files with non-english caption",
    both_caption_count BIGINT COMMENT "number of files with both captions",
    depict_total_count BIGINT COMMENT "number of files with depicts",
    license_total_count BIGINT COMMENT "number of files with license"
)
'''

In [4]:
hive.run(create_table_query.format(
            table_name = overall_table
))

### Overall structured data count

In [8]:
overall_query = '''
WITH captions AS (
    SELECT
        id, 
        ARRAY_CONTAINS(MAP_KEYS(labels),'en') AS en_caption, -- check if has english caption
        SIZE(labels) AS captions_count
    FROM structured_data.commons_entity 
    WHERE snapshot = "{snapshot}"
),

depict_license AS (
    SELECT
        id, 
        SUM(IF(st.mainsnak.property = "P180", 1, 0)) AS depicts_count, -- P180 depicts property
        SUM(IF(st.mainsnak.property = "P275", 1, 0)) AS license_count  -- P275 license property
    FROM structured_data.commons_entity 
         LATERAL VIEW explode(statements) exploded_table as st
     WHERE snapshot = "{snapshot}"
     GROUP BY id
),

total_sdc AS (
    SELECT
        ce.id,
        en_caption,
        captions_count,
        depicts_count,
        license_count
   FROM structured_data.commons_entity ce
      LEFT JOIN captions c ON ce.id = c.id
      LEFT JOIN depict_license dl ON ce.id = dl.id
    WHERE snapshot = "{snapshot}"
),

sdc_counts AS (
    SELECT
        '{wmf_snapshot}' AS month,
        COUNT(DISTINCT(id)) AS sdc_total_count,
        SUM(CASE WHEN captions_count>0 THEN 1 END) AS caption_total_count,
        SUM(CASE WHEN en_caption THEN 1 END) AS en_caption_count,
        SUM(CASE WHEN ((en_caption AND captions_count >1) OR (!en_caption AND captions_count > 0)) THEN 1 END) AS non_en_caption_count,
        SUM(CASE WHEN en_caption AND captions_count >1 THEN 1 END) AS both_caption_count,
        SUM(CASE WHEN depicts_count >0 THEN 1 END) AS depict_total_count,
        SUM(CASE WHEN license_count >0 THEN 1 END) AS license_total_count
    FROM total_sdc        
),

commons_content AS (
    SELECT 
        snapshot AS month,
        COUNT(DISTINCT page_id) AS commons_total_count
FROM wmf.mediawiki_page_history
WHERE snapshot = '{wmf_snapshot}'
    AND wiki_db = 'commonswiki'
    AND NOT page_is_deleted AND NOT page_is_redirect
    AND page_namespace_is_content
GROUP BY snapshot
)

INSERT INTO {aggregate_table}
SELECT
    CONCAT(c.month,'-01') AS month, 
    commons_total_count,
    sdc_total_count,
    caption_total_count,
    en_caption_count,
    non_en_caption_count,
    both_caption_count,
    depict_total_count,
    license_total_count
FROM sdc_counts s 
JOIN commons_content c ON s.month = c.month
'''

In [10]:
hive.run(overall_query.format(
    wmf_snapshot = wmf_snapshot,
    snapshot = entity_snapshot,
    aggregate_table = overall_table
))

,_col0,_col1,_col2,_col3,_col4,_col5,_col6,_col7,_col8
